In [1]:
import pandas as pd
import numpy as nb
from bs4 import BeautifulSoup
import requests
import json
import csv
import time


In [2]:
# Load config files 
def load_json():
    with open('./config.json','r')as file:
        data=json.load(file)
    return data


def construct_url_for_search(search_content,search_engine,num_of_pages):
    search_content=search_content.replace(' ','+')
    # print(search_content,search_engine)
    if search_engine=='Google':
        return f'https://news.google.com/search?q={search_content}'
    elif search_engine=='Yahoo':
        return f'https://news.search.yahoo.com/search?q={search_content}'
    elif search_engine=='Bing':
        return f'https://www.bing.com/news/search?q={search_content}'
    return None

def extract_articles(soup,engine):
    article_lst=[]
    if engine=="Google":
        articles= soup.find_all('article',{"class":"IFHyqb DeXSAc"})
        for article in articles:
            link_of_news=article.find('a',{'class':'WwrzSb'})['href']
            title=article.find('a',{'class':'JtKRv'}).text
            timestamp=article.find('time')
            data_time_value=timestamp['datetime']
            media_name=article.find('div',{'class':'vr1PYe'}).text

            article_lst.append((link_of_news,title,data_time_value.split("T")[0],media_name))
        # pass
            
        
        
    elif engine=="Yahoo":
        articles= soup.find_all('div',{"class":"dd NewsArticle"})
        # print(articles)
        for article in articles:
        
            title_=article.find('h4',{'class':'s-title fz-20 lh-m fw-500 ls-027 mt-8 mb-2'})
            link_of_news=title_.find('a')['href']
            title=title_.find('a').text
            data_time_value=article.find('span',{'class':'s-time fz-14 lh-18 fc-dustygray fl-l mr-4'}).text   
            media_name=article.find('span',{'class':'s-source fw-l'}).text
            article_lst.append((link_of_news,title,data_time_value,media_name))
        pass
        

    elif engine=='Bing':
        articles= soup.find_all('div',{"class":"news-card newsitem cardcommon"})
        for article in articles:
           
           link_of_news=article.find('a',{'class':'title'})['href']
           title=article.find('a',{'class':'title'}).text
           data_time_value=article.find('span',{'aria-label':True}).text 
           media_name_=article.find('div',{'class':'source set_top'})
           if media_name_:
                # Check if there's an image inside the media name
                img_tag = media_name_.find('img')
                if img_tag and img_tag.has_attr('title'):
                    media_name = img_tag['title']
                else:
                    # Otherwise, use the text content
                    media_name = media_name_.text.strip()
           else:
                media_name = 'Unknown' 
           
           article_lst.append((link_of_news,title,data_time_value,media_name))
           
        pass
    
    return article_lst

def save_to_csv(data,file_name='news_results.csv') :
    with open(file_name,mode='w',newline='',encoding='utf-8')as file:
        writer=csv.writer(file)
        writer.writerow(["search_content","search_engine","link_of_news","title","timestamp","media_name"])
        writer.writerows(data)
        print(f'Data successfully saved in {file_name}')
    


def fetch_serching_result(url):
    response=requests.get(url)
    data=BeautifulSoup(response.text,'html.parser')
    # print(data)
    return data


def main():
    
    config=load_json()
    # print(file)
    companies_name=config['company_names']
    keyword_name=config['keywords']
    number_of_pages=config['Number_of_Pages']
    search_engines=config['Search_engine']
    # print(companies_name,keyword_name,number_of_pages)
    search_queries=[]
    for company in companies_name:
        for keyword in keyword_name:
            search_query=company +' '+keyword
            # print(search_query)
            search_queries.append(search_query)
    # print(search_queries)
    
    collected_data=[]
    for search_content in search_queries:
        for search_engine in search_engines:
            url=construct_url_for_search(search_content,search_engine,number_of_pages)
            soup=fetch_serching_result(url)
            if soup:
                articles_data=extract_articles(soup,search_engine)
                for link_of_news,title,timestamp,media_name in articles_data:
                    collected_data.append((search_content,search_engine,link_of_news,title,timestamp,media_name))
                # print(collected_data)
    save_to_csv(collected_data)
 
            # print(soup.prettify())
            
            # print(url)
            # print(soup)
    end_time=time.time()
            
if __name__=="__main__":
    start_time=time.time()

    main()
    end_time=time.time()
    print(end_time-start_time)
    

Data successfully saved in news_results.csv
25.836772680282593
